In [1]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib

import pickle

#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale

#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel

warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp

import vectorizer
importlib.reload(vectorizer)
from vectorizer import  WeightedTfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import *
from sklearn.metrics import classification_report, confusion_matrix



# Reorganizamos el corpus

In [5]:
import joblib
corpus=joblib.load('../data/processed/amazon/corpus_reordered.final.pck')

In [6]:
corpus

,cleaned_text,Target,Topic,prob_topic,outlier_iqr,new_target
0,book recommend friend finally read year late w...,1,1,0.996960,False,1
1,need question know child love impair identify ...,1,1,0.996156,False,1
2,keep interest way definitely read jacob s moth...,1,1,0.995366,False,1
3,c w gortner favorite author time wow queen rel...,1,1,0.998420,False,1
4,love book like jk alex delaware series problem...,1,1,0.995873,False,1
...,...,...,...,...,...,...
49995,great excepy send gray bowl black bowl ands wh...,0,0,0.006945,False,0
49996,find seller sell 12 filter pad 20 zanyzap orde...,0,0,0.004644,False,0
49997,buy small bowl toy poodle s hold easy wash sta...,0,0,0.004846,False,0
49998,little dog like flip cat bowl wet food reason ...,0,0,0.003512,False,0


In [7]:
corpusnormalizado = corpus.iloc[:,[0,5]]
corpusnormalizado.head()

,cleaned_text,new_target
0,book recommend friend finally read year late w...,1
1,need question know child love impair identify ...,1
2,keep interest way definitely read jacob s moth...,1
3,c w gortner favorite author time wow queen rel...,1
4,love book like jk alex delaware series problem...,1


In [8]:
corpussinoutliers = corpus.iloc[:,[0,2]]
corpussinoutliers.head()

,cleaned_text,Topic
0,book recommend friend finally read year late w...,1
1,need question know child love impair identify ...,1
2,keep interest way definitely read jacob s moth...,1
3,c w gortner favorite author time wow queen rel...,1
4,love book like jk alex delaware series problem...,1


In [9]:
topics0 = joblib.load('../objects/amazon/topics0')
topics1 = joblib.load('../objects/amazon/topics1')
feature_names = joblib.load('../objects/amazon/feature_names.final')

In [10]:
longitud = 100
vocab=topics1[-100:]
vocab2 = topics0[-200:]
contador = 0
for word in vocab2:
    if word not in vocab:
        contador = contador+1
        vocab.append(word)
        if contador == longitud:
            break

In [11]:
vocab

['word',
 'romance',
 'definitely',
 'young',
 'star',
 'follow',
 'bite',
 'begin',
 'line',
 'amaze',
 'review',
 'leave',
 'day',
 'different',
 'disappoint',
 'hard',
 'buy',
 'couldn',
 'history',
 'highly',
 'recipe',
 'real',
 'mystery',
 'friend',
 'long',
 'favorite',
 'hope',
 'understand',
 'turn',
 'action',
 'man',
 'try',
 'information',
 'forward',
 'tell',
 'twist',
 'make',
 'finish',
 'need',
 'take',
 'family',
 'work',
 'old',
 'easy',
 'excellent',
 'help',
 'wonderful',
 'learn',
 'live',
 'child',
 'happen',
 'reader',
 'world',
 'give',
 'don',
 'didn',
 'plot',
 'thing',
 'year',
 'new',
 'people',
 'little',
 'movie',
 'be',
 'keep',
 'fun',
 'page',
 'come',
 'novel',
 'start',
 'get',
 'go',
 'lot',
 'feel',
 'have',
 'look',
 'wait',
 'life',
 'way',
 'recommend',
 'well',
 'want',
 'know',
 'find',
 'think',
 'time',
 'end',
 'interest',
 'author',
 'enjoy',
 'write',
 'like',
 'character',
 'good',
 'great',
 'series',
 'love',
 'story',
 'read',
 'book',

In [12]:
wtfidf = WeightedTfidfVectorizer(vocabulary = vocab) #aquí ya hemos hecho una selección inicial de variables

In [13]:
newdtm = wtfidf.fit_transform(corpusnormalizado)

In [14]:
newdtm

<50000x200 sparse matrix of type '<class 'numpy.float32'>'
	with 456854 stored elements in Compressed Sparse Row format>

In [16]:
newdtmso = wtfidf.fit_transform(corpussinoutliers)

In [17]:
newdtmso

<50000x200 sparse matrix of type '<class 'numpy.float32'>'
	with 456854 stored elements in Compressed Sparse Row format>

In [18]:
joblib.dump(vocab,'../objects/amazon/vocab.dtm.final')
joblib.dump(newdtm,'../objects/amazon/newdtm.final')

['../objects/amazon/newdtm.final']

In [19]:
joblib.dump(newdtmso,'../objects/amazon/newdtmsinoutliers.final')

['../objects/amazon/newdtmsinoutliers.final']

In [20]:
joblib.dump(corpusnormalizado,'../data/processed/amazon/corpus_normalizado.final')

['../data/processed/amazon/corpus_normalizado.final']

In [21]:
joblib.dump(corpussinoutliers,'../data/processed/amazon/corpus_normalizado_sin_outliers.final')

['../data/processed/amazon/corpus_normalizado_sin_outliers.final']